In [2]:
# PART 0: IMPORTING LIBRARIES

import sys
import os

import pandas as pd
import numpy as np

import math as m
import datetime
from collections import defaultdict

from sklearn import linear_model
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

from sklearn.model_selection import KFold

import pickle

In [18]:
# PART 1: CREATE DATE FILE

# Reads in 'cut-off date' for data to be used to train 
# currdate = sys.argv[1]
currdate = pd.to_datetime('15/8/2021', dayfirst = True)
iterdate = pd.to_datetime('03/07/2017', dayfirst = True)
FIRSTDATE = pd.to_datetime('03/07/2017', dayfirst = True)


day = datetime.timedelta(days=1)

datematchweek = dict()
week = 1
count = 1
while iterdate <= currdate:
    if count == 8:
        count = 1
        week += 1
    
    datematchweek[iterdate.strftime('%d/%m/%Y')] = week
    iterdate += day
    
    count += 1

dates = list(datematchweek.keys())
weekno = list(datematchweek.values())

# Make the dictionary of dates to week number into a dataframe
DatesToWeek_DF = pd.DataFrame({'dates': dates, 'weekno':weekno})

DatesToWeek_DF.to_csv('../DateMatchWeek.csv', index = False)

# Record the week number of the cut-off date
thisweek = max(weekno)


In [33]:
%%time
# # PART 2: WRANGLING ORIGINAL DATA

#### FUTURE UPDATE: towranglelist1 stores records which have dates in format [d]d/[m]m/yyyy; towranglelist2 stores records which have dates in format yyyy-mm-dd. Need to update lists accordingly
towranglelist1 = ['action.csv', 'competency_record.csv', 'form_record.csv', 'incident.csv', 'users.csv','assets.csv', 'form_template.csv']
towranglelist2 = ['users.csv'] 
# because users need to be wrangled in terms of employees and inductioneers. 3 is inductionuser 4 is employees

def wrangle(filename, datedata, mode):
    """ cleans the file. 4 modes for four different ways to clean the data - all pretty similar except mode 3 and 4 selects users of particular hr types, and mode 2 deals with dates of a different format """
    data = pd.read_csv(f"./Data/{filename}")
    data = data[(data['domain'] != 'cruse') & (data['domain'] != 'demo') & (data['domain'] != 'demo_2')]
    
    # First drop: get rid of rows from domains demo and demo_2       
    if mode == 2:
        data = data[data['hr_type'] != 'InductionUser']
        req = ['InductionUser']
    
    elif mode == 3:
        data = data[(data['hr_type'] != 'Casual') & (data['hr_type'] != 'Employee') & (data['hr_type'] != 'Subcontractor')]
    
    # re-setup date dictionary from the DataFrame
    dates = list(datedata['dates'])
    weekno = list(datedata['weekno'])
    
    datematchdict = {dates[i]: weekno[i] for i in range(len(dates))}
    
    data['created_at'] = pd.to_datetime(data['created_at'], errors = 'coerce', dayfirst = True)
    
    data = data[(data['created_at'] >= FIRSTDATE) & (data['created_at'] <= currdate)]
    
    data['created_at'] = pd.to_datetime(data['created_at'], dayfirst = True)
    
    data.index = range(len(data))
    
    data['created_at'] = data['created_at'].dt.strftime('%d/%m/%Y')
    
    actweekno = [datematchdict[data.loc[i]['created_at']] for i in range(len(data))]
    
    
    groupup = defaultdict(int)
    
    for i in range(len(data)):
        groupup[f"{data.loc[i]['domain']} {actweekno[i]}"] += 1
    
    groupupkey = list(groupup.keys())
    groupupval = list(groupup.values())
                 
    # create lists that contain just domain name and week number
    out1 = [groupupkey[i].split()[0] for i in range(len(groupupkey))]
    out2 = [groupupkey[i].split()[1] for i in range(len(groupupkey))]
    
    # export the wrangled file as a csv (each of these files are wrangled version of the raw data files (of each of the client's recorded activity in lucidity) in terms of counts per week per domain)
    out = pd.DataFrame({'Domain': out1, 'Week': out2, 'COUNT': groupupval})
    
    if mode == 1:
        out.to_csv(f'./Partial_Output/_2_{filename.split(".")[0]}_clean.csv', index = False)
        out.to_csv(f'../History/Week {thisweek}/Partial_Output/_2_{filename.split(".")[0]}_clean.csv', index = False)
        
    elif mode == 2:
        out.to_csv('./Partial_Output/_2_Users_Inductee_clean.csv', index = False)
        out.to_csv(f'../History/Week {thisweek}/Partial_Output/_2_{filename.split(".")[0]}_clean.csv', index = False)
        
    else:
        out.to_csv('./Partial_Output/_2_Users_norm_employee_clean.csv', index = False)
        out.to_csv(f'../History/Week {thisweek}/Partial_Output/_2_{filename.split(".")[0]}_clean.csv', index = False)
    
# OS housekeeping and running each of the files through wrangle()
if not os.path.exists('./Partial_Output'):
    os.mkdir('./Partial_Output')

if not os.path.exists(f'../History/Week {thisweek}/Partial_Output'):
    os.makedirs(f'../History/Week {thisweek}/Partial_Output')
        
for file in towranglelist1:
    wrangle(file, DatesToWeek_DF, 1)
    
for file in towranglelist2:
    wrangle(file, DatesToWeek_DF, 2)
    
for file in towranglelist2:
    wrangle(file, DatesToWeek_DF, 3)

<timed exec>:78: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
<timed exec>:78: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
<timed exec>:78: DtypeWarning: Columns (5,6,7,8,16) have mixed types.Specify dtype option on import or set low_memory=False.
<timed exec>:81: DtypeWarning: Columns (5,6,7,8,16) have mixed types.Specify dtype option on import or set low_memory=False.
<timed exec>:84: DtypeWarning: Columns (5,6,7,8,16) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 18min 23s, sys: 4.66 s, total: 18min 28s
Wall time: 18min 28s


In [ ]:
# PART 3: COMBINE PREVIOUSLY WRANGLED DATAFRAMES INTO ONE (FILLING IN WEEKS WITH NO ACTIVITY)

# import all cleaned data
asset = pd.read_csv('./Partial_Output/_2_assets_clean.csv')
actions = pd.read_csv('./Partial_Output/_2_action_clean.csv')
competency = pd.read_csv('./Partial_Output/_2_competency_record_clean.csv')
form_record = pd.read_csv('./Partial_Output/_2_form_record_clean.csv')
form_templates = pd.read_csv('./Partial_Output/_2_form_template_clean.csv')
incidents = pd.read_csv('./Partial_Output/_2_incident_clean.csv')
users = pd.read_csv('./Partial_Output/_2_users_clean.csv')
users_induct = pd.read_csv('./Partial_Output/_2_users_Inductee_clean.csv')
users_norm_emp = pd.read_csv('./Partial_Output/_2_users_norm_employee_clean.csv')

# Find a set of the domain names - for finding the "earliest recorded date" of activity/usage for each
set1 = set(asset['Domain'])
set2 = set(actions['Domain'])
set3 = set(competency['Domain'])
set4 = set(form_record['Domain'])
set5 = set(form_templates['Domain'])
set6 = set(incidents['Domain'])
set7 = set(users['Domain'])

fullset = set1.union(set2).union(set3).union(set4).union(set5).union(set6).union(set7)
fullsetlist = list(fullset) # Now have a full set of the domains
fullsetlist.sort()

iteration = [asset, actions, competency, form_record, form_templates, incidents, users]
newiteration = [asset, actions, competency, form_record, form_templates, incidents, users, users_induct, users_norm_emp]

# Find first week recorded and put them in a dictionary of (key: value) = (domain name: first week of activity)
startweek = dict()

for data in newiteration:
    dom = list(data['Domain'])
    week = list(data['Week'])
    count = list(data['COUNT'])
    
    for i in range(len(dom)):
        if dom[i] in startweek:
            if week[i] < startweek[dom[i]]:
                startweek[dom[i]] = week[i]
        else:
            startweek[dom[i]] = week[i]
            
startweeklist = list(startweek.items())
startweeklist.sort()

# Create a template for recording the data (now fill out gaps between start week and week 216 where there is 0 data)
combineddatatemplate = dict()

# first initiate a blank dictionary with all weeks from first week of activity to cutoffdate's week
for i in range(len(startweeklist)):
    for j in range(startweeklist[i][1], thisweek+1):
        combineddatatemplate[f'{startweeklist[i][0]} {j}'] = 0

# create blank copies of this initialised template dictionary, and fill them in based on counts from the output of PART 2
assetcomb = combineddatatemplate.copy()
actionscomb = combineddatatemplate.copy()
competencycomb = combineddatatemplate.copy()
form_recordcomb = combineddatatemplate.copy()
form_templatescomb = combineddatatemplate.copy()
incidentscomb = combineddatatemplate.copy()
userscomb = combineddatatemplate.copy()
users_inductcomb = combineddatatemplate.copy()
users_norm_empcomb = combineddatatemplate.copy()

dictlist = [assetcomb, actionscomb, competencycomb, form_recordcomb, form_templatescomb, incidentscomb, userscomb, users_inductcomb, users_norm_empcomb]

# Now fill in the details where there are records (because all dictionary slots initialised, only need to repalce data for weeks where there was a count recorded, and all other are fine to be left untouched - just ends up being 0)
for k in range(len(dictlist)):
    dom = list(newiteration[k]['Domain'])
    week = list(newiteration[k]['Week'])
    count = list(newiteration[k]['COUNT'])
    
    for i in range(len(dom)):
        dictlist[k][f'{dom[i]} {week[i]}'] = count[i]

uniqueid = list(assetcomb.keys())
assetcount = list(assetcomb.values())
actioncount = list(actionscomb.values())
competencycount = list(competencycomb.values())
form_recordcount = list(form_recordcomb.values())
form_templatescount = list(form_templatescomb.values())
incidentscount = list(incidentscomb.values())
userscount = list(userscomb.values())
users_inductcount = list(users_inductcomb.values())
users_norm_empcount = list(users_norm_empcomb.values())

# create two more lists that contain just domain and just week - maximises chances of making future wrangling easier
doms = []
weekss = []

for i in range(len(uniqueid)):
    doms.append(uniqueid[i].split()[0])
    weekss.append(uniqueid[i].split()[1])
    
# Create a new column for counting the number of weeks since particular company started at Lucidity
selfweeks = []
count = 0

# (logic of loop fairly simple - if domain column runs into new company then reset the count)
prev = doms[0]
for i in range(len(doms)):
    if doms[i] == prev:
        count += 1
        selfweeks.append(count)
    else:
        count = 1
        selfweeks.append(count)
    prev = doms[i]

# Create a prelim score of sum of the three attributes to be used in determining a function
baseactscore1 = list()
for i in range(len(uniqueid)):
    baseactscore1.append(form_recordcount[i]+competencycount[i]+users_inductcount[i])

# turn it into one dataframe and output
out = pd.DataFrame({'ID': uniqueid, 'Domain': doms, 'Week': weekss, 'Selfweeks': selfweeks,
                    'Assets': assetcount, 'Actions': actioncount, 'Competency': competencycount, 
                    'Form_record': form_recordcount, 'Form_template': form_templatescount,
                   'Incident': incidentscount, 'Users': userscount, 
                    'Users_induction': users_inductcount, 'Users_norm_emp': users_norm_empcount, 
                    "Prelim_action_score": baseactscore1})

# This file now has counts of all activities grouped by week by client/domain, sorted by domian and clients.  
out.to_csv("./Partial_Output/_3_combined_cleaned_data.csv", index = False)
out.to_csv(f"../History/Week {thisweek}/Partial_Output/_3_combined_cleaned_data.csv", index = False)